 ## BERT Model ## 

In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import concurrent.futures

# Function to load the tokenizer with a timeout
def load_tokenizer_with_timeout(model_name, timeout=30):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(BertTokenizer.from_pretrained, model_name)
        try:
            tokenizer = future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print("Timeout while loading tokenizer. Skipping.")
            return None
        except Exception as e:
            print(f"Error while loading tokenizer: {e}")
            return None
    return tokenizer

# Function to load the model with a timeout
def load_model_with_timeout(model_name, num_labels, timeout=30):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(BertForSequenceClassification.from_pretrained, model_name, num_labels=num_labels)
        try:
            model = future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print("Timeout while loading model. Skipping.")
            return None
        except Exception as e:
            print(f"Error while loading model: {e}")
            return None
    return model

# Load the dataset
data = pd.read_csv('Police_Department_Incidents_-_Previous_Year__2016_.csv')

# Data Preprocessing
data.dropna(subset=['PdDistrict'], inplace=True)  # Drop rows with missing PdDistrict
data = data[data['Category'].notna()]  # Drop rows with missing Category


data = data.sample(1000)  

# Load tokenizer and model with timeout
tokenizer = load_tokenizer_with_timeout('bert-base-uncased')
if tokenizer is None:
    raise RuntimeError("Tokenizer could not be loaded. Exiting.")

# Prepare labels
label_to_id = {label: i for i, label in enumerate(data['Category'].unique())}
data['label_id'] = data['Category'].map(label_to_id)

# Prepare text data for BERT
def encode_data(texts, labels, max_length=64):  # Reduced max_length for faster processing
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')
    labels = torch.tensor(labels.tolist())
    return encodings, labels

# Encode the dataset
X_train, X_test, y_train, y_test = train_test_split(data['Descript'], data['label_id'], test_size=0.2, random_state=42)
train_encodings, train_labels = encode_data(X_train, y_train)
test_encodings, test_labels = encode_data(X_test, y_test)

# Load the model
model = load_model_with_timeout('bert-base-uncased', num_labels=len(label_to_id))
if model is None:
    raise RuntimeError("Model could not be loaded. Exiting.")

# Define a dataset class
class CrimeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CrimeDataset(train_encodings, train_labels)
test_dataset = CrimeDataset(test_encodings, test_labels)

# Training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduce the number of epochs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save model at the end of each epoch
    save_total_limit=2,  # Save only the last two checkpoints
    load_best_model_at_end=True,  # Load the best model when finished training
    fp16=False,  # Disable mixed precision for CPU compatibility
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

try:
    trainer.train()
except KeyboardInterrupt:
    print("Training interrupted. Saving partial model...")
    trainer.save_model("interrupted_model_save")
    tokenizer.save_pretrained("interrupted_tokenizer_save")

# Save the model and tokenizer
model_path = 'bert_crime_model.bin'
tokenizer_path = 'bert_crime_tokenizer'

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

# Prediction and Evaluation
preds = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(preds.predictions), axis=1)

# Print classification report
from sklearn.metrics import classification_report
print("BERT Classification Report:")
print(classification_report(y_test, y_pred))

C:\Users\0860\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'loss': 3.3888, 'learning_rate': 2.5e-06, 'epoch': 0.05}


 10%|█         | 20/200 [00:32<05:02,  1.68s/it]

{'loss': 3.3985, 'learning_rate': 5e-06, 'epoch': 0.1}


...

Model weights saved in ./results\checkpoint-200\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-200 (score: 0.8584941625595093).
100%|██████████| 200/200 [05:02<00:00,  1.51s/it]
Configuration saved in bert_crime_model.bin\config.json


{'train_runtime': 302.2254, 'train_samples_per_second': 2.647, 'train_steps_per_second': 0.662, 'train_loss': 2.247146849632263, 'epoch': 1.0}


Model weights saved in bert_crime_model.bin\pytorch_model.bin
tokenizer config file saved in bert_crime_tokenizer\tokenizer_config.json
Special tokens file saved in bert_crime_tokenizer\special_tokens_map.json
***** Running Prediction *****
  Num examples = 200
  Batch size = 4
100%|██████████| 50/50 [00:06<00:00,  7.47it/s]

BERT Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       0.98      1.00      0.99        57
           3       0.20      1.00      0.33         1
           4       0.79      1.00      0.88        11
           5       0.85      1.00      0.92        11
           6       0.73      0.79      0.76        24
           7       0.00      0.00      0.00         3
           8       1.00      1.00      1.00        26
           9       0.00      0.00      0.00         1
          10       0.69      1.00      0.82        27
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00         4
          14       1.00      1.00      1.00         4
          15       1.00      1.00      1.00         1
          16       0.78      0.78      0.78         9
          17       0.00      0.00      0.00         1

C:\Users\0860\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\0860\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\0860\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",